In [10]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


['..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/tmp/tmpm_an8fb8']
['..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.i

In [ ]:
# models = [
#     {"path":"/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/save_checkpoint_7_thu.pth",
#     "model_size":"base","model_name":"base_all_e_7"},
#     {"path":"/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/Retrieval_aliproduct2_filtered/save_checkpoint_4.pth",
#     "model_size":"base","model_name":"base_1st_e_5"},
#     {"path":"/home/ubuntu/Desktop/base_2nd/base_2nd_e_5.pth","model_size":"base","model_name":"base_2nd_e_5"},
#     {"path":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_5.pth",
#     "model_size":"base","model_name":"base_3rd_e_5"},
#      {"path":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_8.pth","model_size":"large","model_name":"large_all_e_9"},
#     {"path":"/home/ubuntu/Desktop/large_v4/save_checkpoint_4.pth","model_size":"large","model_name":"large_v1_e_5"},
#     ]


models = [
       {"path":"/home/ubuntu/Desktop/large_v7/save_checkpoint_0.pth","model_size":"large","model_name":"large_v4_e_1"}

    ]


test_df_path = "/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference/test.csv"



col_to_test =  "test_caption"

In [15]:
image_size = 384
preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 


In [16]:
def gen_embed(caption_col,clip_model):
    test_loader,df = prepare_data(test_df_path,
    CONFIG.test_image_data_dir,"test_imgs"
    ,"test_imgs",caption_col,160,preprocess,CONFIG.global_random_state,test=True,tokenize=False)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred

    return image_embed,text_embed

In [17]:
def save_embed(image_embed,text_embed,model_size:str,prefix:str):
    img_embed_path = "./image_embeds"
    txt_embed_path = "./text_embeds"
    img_embed_filename = f"img_emded-{model_size}-{prefix}.pkl"
    txt_embed_filename = f"txt_emded-{model_size}-{prefix}.pkl"
    img_embed_full_path = f"{img_embed_path}/{img_embed_filename}"
    txt_embed_full_path = f"{txt_embed_path}/{txt_embed_filename}"

    image_embed_info_dict = {"model_name":prefix,"model_size":model_size,"pair_name":txt_embed_filename,"embed":image_embed}
    text_embed_info_dict = {"model_name":prefix,"model_size":model_size,"pair_name":img_embed_filename,"embed":text_embed}

    with open(img_embed_full_path,"wb") as file:
        print(f"saving img embed from {prefix} to {img_embed_full_path}")
        pickle.dump(image_embed_info_dict,file)
    with open(txt_embed_full_path,"wb") as file:
        print(f"saving txt embed from {prefix} to {txt_embed_full_path}")
        pickle.dump(text_embed_info_dict,file)
    

In [18]:

for model in models:
    print(f"extracting embeddings from "+model["model_name"])
    clip_model =ALIPRODUCT_BLIP(model["path"],image_size,vit=model["model_size"])
    image_embed,text_embed = gen_embed(col_to_test,clip_model)
    text_embed_no_null = text_embed[0:20000]
    save_embed(image_embed,text_embed_no_null,model["model_size"],model["model_name"])
    


extracting embeddings from large_v4_e_1


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


load checkpoint from /home/ubuntu/Desktop/large_v7/save_checkpoint_0.pth


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [52:22<00:00, 10.04s/it]
saving img embed from large_v4_e_1 to ./image_embeds/img_emded-large-large_v4_e_1.pkl
saving txt embed from large_v4_e_1 to ./text_embeds/txt_emded-large-large_v4_e_1.pkl
